<a href="https://colab.research.google.com/github/ayush5harma/ViewshedAnalysis/blob/master/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import random
from bresenham import bresenham
import csv
import subprocess, pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
fn = 'DEM_30m_whole_area.tif'

from numpy import load

rasterArray = np.array(load('dem.npy'))
#dem.npy file and buffer are needed for running this code


In [ ]:
size_x=rasterArray.shape[0]
size_y=rasterArray.shape[1]

In [ ]:
def switch(k):
    ps = {
        1: 'CPU-R2-',
        2: 'GPU-R2-',
        3: 'CPU-R3-',
        4: 'GPU-R3-'
    }
    return ps.get(k, 'Invalid Choice')



In [ ]:
def viewshed_gpu(Visiblity, obs_X, obs_Y, path, radius, dem_x, dem_y, type):
    inp = path + '\n' + str(dem_x) + '\n' + str(dem_y) + '\n' + str(radius) + '\n' + str(obs_X) + '\n' + str(obs_Y) + '\n' + str(type) + '\n'
    res = bytes(inp, 'utf-8')
    process = subprocess.Popen("Release/RasterViewshed.exe", stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    std = process.communicate(input=res)[0]
    ps = switch(type)
    pa = ps + str(obs_X) + '-' + str(obs_Y) + '.csv'
    pa1 = ps + str(obs_X) + '-' + str(obs_Y) + '.bmp'
    pk='1sat'+ str(obs_X) + '-' + str(obs_Y) + '.csv'
    df = pd.DataFrame()
    view = np.genfromtxt(pa, delimiter=",")
    view = view[:, :-1]
    for i in range(size_x):
        for j in range(size_y):
            if view[i, j]==1:
                d={
                    'x':i,
                    'y':j
                }
                df=df.append(d,ignore_index=True)
    
    df.to_csv(pk)
    
    for d in df.index:
        x=int(df['x'][d])
        y=int(df['y'][d])
        Visiblity[int(x),int(y)]=1
    os.remove(pa1)
    os.remove(pa)
    return Visiblity

In [ ]:
ridgeline_buffer=np.load("Buffer")
ridgeline_buffer=ridgeline_buffer.tolist()  #here bufferridgeline_buffer refers to the Ridgeline-buffer obtained by running the ridgeline-buffer code first. The values once obtained can be saved as a spydata file for future use.
no_of_ridgeline_points = len(ridgeline_buffer)

no_of_iterations = int(input("enter the number of iterations"))

no_of_particles = int(input("enter the number of particles"))  #no. of chromosomes to be initialized in the population

no_of_viewpoints = int(input("enter the number of viewpoints"))

radius = int(input("enter the radius of coverage by viewpoints"))  #Should be given in terms of pixels. Example:- radius = 500 means radius of 500 pixels which equals 500*30 = 15km approx.

crossover_rate = float(input("enter the crossover rate"))   #a Floating point value between 0 and 1. Example:- 0.8 means the new generation would have 80% new chromosomes obtained by crossover.

mutation_rate = float(input("enter the mutation rate"))    #a Floating point valuebetween 0 and 1. Example:- 0.01 means that any given chromosome has a 1% chance of being mutated.

ratio_of_ridgebuffer_population = float(input("enter the ratio of ridgeline population"))   #a Floating point valuebetween 0 and 1. Example:- 0.25 means 25% of the initial population would be initialized from the ridgeline-buffer.

height_of_sensor = int(input("enter the height of sensor in metres"))

Global_best_particle = np.zeros((2*no_of_viewpoints)+1)  #it keeps track of the best solution yet obtained at any given point in time.

Final_visibility_matrix = np.zeros((size_x,size_y))

Best_Particle_by_Epoch = np.zeros((no_of_iterations, (2*no_of_viewpoints)+1))


In [ ]:
current_gen=load("intial_pop.npy")
#this command requires you to use intial_pop.npy file in directory

In [ ]:
def Fitness(current_generation, global_best_particle):
    
    for i in range(no_of_particles):
        
        visibility_matrix = np.zeros((size_x,size_y))   #used to keep track of all the raster cells in the viewshed of any of the censors in a chromosome.
        
        for j in range(no_of_viewpoints):
            ps='1sat'+str(int(current_generation[i][2*j]))+'-'+str(int(current_generation[i][2*j+1]))+'.csv'
            filelist=[f for f in os.listdir(str(os.getcwd())) if f==ps]
            if len(filelist)==1:
                view = pd.read_csv(ps)
                for d in view.index:
                    x=view['x'][d]
                    y=view['y'][d]
            
                    visibility_matrix[int(x),int(y)]=1
                                     
                
            else:
                visibility_matrix = viewshed_gpu(visibility_matrix, int(current_generation[i][2*j]), int(current_generation[i][2*j+1]),fn,radius,size_x,size_y,2)
            
        
        count = 0                                #finding the number of pixels visible.
        for k in range(size_x):
            for l in range(size_y):
                if (visibility_matrix[k][l] == 1):
                    count = count+1
                    
        percentage_covered = count/(size_x*size_y)*100
        
        current_generation[i][2*no_of_viewpoints] = percentage_covered
        
    sorted_current_generation = current_generation[current_generation[:,(2*no_of_viewpoints)].argsort()[::-1]]    #sorting the current generation according to fitness.
    
        
    new_Global_best_particle = global_best_particle
    
    if (global_best_particle[2*no_of_viewpoints] < sorted_current_generation[0][2*no_of_viewpoints]):          #checking for a new global best.
        new_Global_best_particle = sorted_current_generation[0]
        
        
    return sorted_current_generation, new_Global_best_particle, current_generation



#%%


def Selection(Current_generation):    #Stochastic Universal Sampling is used for selection of parents.
    
    no_of_parents = int(float(crossover_rate * no_of_particles))   #determining the number of parents to be selected for crossover.
    
    sum_of_fitness = 0
    
    for i in range(no_of_particles):
        sum_of_fitness =  sum_of_fitness + Current_generation[i][2*no_of_viewpoints]
        
    distance_between_pointers = sum_of_fitness / no_of_parents
    
    n = random.random() * sum_of_fitness                      #generating a random pointer.
    
    pointer_array = np.zeros(no_of_parents)
   
    pointer_array[0] = n
    
    for i in range(no_of_parents-1):
        pointer_array[i+1] = pointer_array[i] + distance_between_pointers 
        
    mating_pool = np.zeros(no_of_parents)
    
    for i in range(no_of_parents):
        
        ps = 0
        c = -1
        
        while (ps < (pointer_array[i] % sum_of_fitness)):
            ps = ps + Current_generation[c][2*no_of_viewpoints]
            c = c+1
            
        mating_pool[i] = c
        
    np.random.shuffle(mating_pool)                  #mating pool is shuffled to make crossover happen between random parents in the crossover step.
        
    return mating_pool                              #mating pool consists of the indices of the chromosomes/parents selected from the current generation for crossover.       
    
        

#%%


def Crossover(Current_Generation,Mating_Pool):      #The top chromosomes are carried on as they are from one generation to the other, Rest are replaced by offsprings.
    
    new_gen = Current_Generation
    
    t = len(Mating_Pool)
    n = no_of_particles - t
    
    for i in range(0,t,2):                           #since mating pool was shuffled in selection step, consecutive parents crossover is equivalent to random parents mating.
        
        for j in range(no_of_viewpoints):
            
            a = random.randint(0,1)                 #if a =1 then first parent gives gene to first offspring, else if a = 0 then first parent gives gene to second offspring
            
            if (a==1):
                new_gen[n+i][2*j] = Current_Generation[int(Mating_Pool[i])][2*j]
                new_gen[n+i][2*j+1] = Current_Generation[int(Mating_Pool[i])][2*j+1]
                
                new_gen[n+i+1][2*j] = Current_Generation[int(Mating_Pool[i+1])][2*j] 
                new_gen[n+i+1][2*j+1] = Current_Generation[int(Mating_Pool[i+1])][2*j+1] 
                
            else:
                new_gen[n+i][2*j] = Current_Generation[int(Mating_Pool[i+1])][2*j]
                new_gen[n+i][2*j+1] = Current_Generation[int(Mating_Pool[i+1])][2*j+1]
                
                new_gen[n+i+1][2*j] = Current_Generation[int(Mating_Pool[i])][2*j] 
                new_gen[n+i+1][2*j+1] = Current_Generation[int(Mating_Pool[i])][2*j+1]
                
    
    return new_gen                     #consists of the top chromosomes from the last generation and the new offsprings created.


#%%



def Mutation(current__generation):       
    
    new_mutated_gen = current__generation
    
    for i in range(1,no_of_particles):
        
        b = random.random()
        
        if (b <= mutation_rate):
            print("mutated")
            
            for a in range(no_of_viewpoints):                          #if any chromosome of the generation is mutated, all of its viewpoints are randomly placed in the buffer area.
                
                rb = random.randint(0,no_of_ridgeline_points-1)
                new_mutated_gen[i][2*a] =  ridgeline_buffer[rb][0]
                new_mutated_gen[i][2*a+1] = ridgeline_buffer[rb][1]
                
            
    return new_mutated_gen




#%%

#running all the iterations for the GA

In [ ]:
import time
for i in range(51,100):
    start=time.time()
    
    current_gen, Global_best_particle, Current_generation = Fitness(current_gen, Global_best_particle) 
    
    Mating_pool = Selection(current_gen)
    
    current_gen = Crossover(current_gen,Mating_pool)
    
    current_gen = Mutation(current_gen)
    
    print("best fitness till iteration ", i+1, " :- ", round(Global_best_particle[2*no_of_viewpoints], 6))
    
    Best_Particle_by_Epoch[i] = Global_best_particle
    print("Iteration Time :-",int((time.time()-start)/3600), " Hrs")
    np.save("Best_particle.npy",Best_Particle_by_Epoch)
    np.save("Current_gen.npy",current_gen)
    
    


#%%    

#Displaying result and making final changes to generate new TIFF file

print("best solution is : - \n" , Global_best_particle)

print("percentage covered is :- " , round(Global_best_particle[2*no_of_viewpoints], 6))

for v in range(no_of_viewpoints):
    x = 0
    y = radius
    
    # Cirlce Points of the final solution generated.

    d = int(3 - (2 * radius))

    while x<= y: 
    
        Final_visibility_matrix[int(Global_best_particle[2*v])+x][int(Global_best_particle[2*v+1])+y] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])-x][int(Global_best_particle[2*v+1])+y] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])+x][int(Global_best_particle[2*v+1])-y] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])-x][int(Global_best_particle[2*v+1])-y] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])+y][int(Global_best_particle[2*v+1])+x] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])-y][int(Global_best_particle[2*v+1])+x] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])+y][int(Global_best_particle[2*v+1])-x] = 1
        Final_visibility_matrix[int(Global_best_particle[2*v])-y][int(Global_best_particle[2*v+1])-x] = 1
    
        if d<0:
            d = d + (4*x) + 6
        else:
            d = d + (4*(x-y)) + 10
            y = y-1
        x = x+1
        
        
    Final_visibility_matrix = Viewshed(Final_visibility_matrix, int(Global_best_particle[2*v]), int(Global_best_particle[2*v+1]) )


#%%
from numpy import save
save('result.npy',Final_visiblity_matrix)

with open('Results/Sensor Positions After Each Epoch.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(Best_Particle_by_Epoch)